In [55]:
import pyodbc 
import pandas as pd
import requests
import datetime
# API key: a3f721054b83f57e9baa

def curr_conv():
    ####   SQL data read from SQL db    ####
    server = 'localhost'
    database = 'AdventureWorks2019'
    username = 'sa'
    password = 'critically9'
    driver='{ODBC Driver 17 for SQL Server}'

    cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PWD='+ password+';UID='+username)
    cursor = cnxn.cursor()
    # select rows from SQL table into dataframe.
    query = """
            SELECT distinct FromCurrencyCode, ToCurrencyCode
            FROM [Sales].[CurrencyRate]
            order by ToCurrencyCode
            """
    df = pd.read_sql(query, cnxn)

    # whole data
    #print(df) 

    # number of rows
    #print(len(df))

    # Specific row and column
    #print (df.loc[0]["ToCurrencyCode"])
    #print (df.loc[1]["ToCurrencyCode"])

    # loop through dataframe
    #for index in df.index:
    #    print (df.loc[index]["FromCurrencyCode"] + '-' + df.loc[index]["ToCurrencyCode"])


    ####   API    ####
    # From and To Both ways
    #response = requests.get("https://free.currconv.com/api/v7/convert?q=USD_PHP,PHP_USD&compact=ultra&apiKey=a3f721054b83f57e9baa")

    # From - One way USD to Other Currency
    #response = requests.get("https://free.currconv.com/api/v7/convert?q=USD_PHP&compact=ultra&apiKey=a3f721054b83f57e9baa")

    base_url = "https://free.currconv.com/api/v7/convert?q=USD_{}&compact=ultra&apiKey=a3f721054b83f57e9baa"
    f = '%Y-%m-%d %H:%M:%S'
    results = []
    for index in df.index:
        from_cc = df.loc[index]["FromCurrencyCode"]
        to_cc = df.loc[index]["ToCurrencyCode"]
        url = base_url.format(to_cc)
        response = requests.get(url)
        #print(response.json())
        cc_key = from_cc+'_'+to_cc
        if cc_key in response.json():
            cc_val = response.json()[cc_key]
        else:
            # value does not exist
            cc_val = 0.0
        now = datetime.datetime.now()

        row = (now.strftime(f),from_cc,to_cc,cc_val,cc_val,now.strftime(f))
        results.append(row)
    
    #print(results)

    #### Insert into SQL database table  ####

    sql_qry = '''   INSERT INTO [AdventureWorks2019].[Sales].[CurrencyRate]                  
                    (CurrencyRateDate,FromCurrencyCode,ToCurrencyCode,AverageRate,EndOfDayRate,ModifiedDate) 
                    VALUES
                    (?,?,?,?,?,?)
                                    '''
    cursor.fast_executemany = True

    cursor.executemany(sql_qry,results)
    cnxn.commit()
    
    print(datetime.datetime.now())
    print("**** Data Insert Completed ****")

    


#curr_conv()        






**** Data Insert Completed ****
